In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import Delaunay, ConvexHull
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, to_rgba
import matplotlib.patches as patches
import os
import scipy.linalg as sclinalg
import copy

In [2]:
'''
Parameters
'''
radius = 1
res = 40
rho = 6.371e6
R = 1.496e11
alpha = 90/180*np.pi
beta = 0/180*np.pi

sigma = 0.01
nwater = 1.33
epsilon = 0.000000001

day = 24
Ndays = 365
year = Ndays*day
omega_day =  2*np.pi/day 
omega_year = 2*np.pi/year


delta_t = 1
delta_phi = np.pi/res
delta_theta = np.pi/res
delta_Omega = delta_phi*delta_theta 

hours = np.arange(0, day)
time_array = hours.copy()

N_ave = 187

for i in range(1, Ndays):
    time_array = np.append(time_array, hours + i*year/Ndays)

time_res = len(time_array)
print(time_res)

8760


In [3]:
'''
creating functions for albedo matrix

'''
def observabledomain(cos_theta_i,cos_theta_r):
    if cos_theta_i >= 0 and cos_theta_r >= 0:
        return 1
    else:
        return 0    
def deltapeak(x,sigma):
    delta = np.exp(-(x)**2/(2*sigma))/np.sqrt(2*np.pi*sigma)
    return delta


def create_sphere1(r, res):
    '''
    creates points on a sphere in the form:
        [[ x1.  y1.  z1.]
         ....
         [ xn.  yn.  zn.]]
    '''
    phi = np.linspace(0, 2*np.pi, 2*res)
    theta = np.linspace(0, np.pi, res)

    theta, phi = np.meshgrid(theta, phi)  
    
    theta_flat = np.ndarray.flatten(theta, order = 'C')
    phi_flat = np.ndarray.flatten(phi, order = 'C')
    
    r_pre = r*np.sin(theta_flat)
    x = np.cos(phi_flat)*r_pre
    y = np.sin(phi_flat)*r_pre
    z = r*np.cos(theta_flat)

    ''' 
    flatten arrays, delete top- end bottom points of circle and append both points once
    '''
#    x = np.delete(x, [0, res-1], 1)
#    x = np.ndarray.flatten(x, order ='F') 
#    x = np.append(x, [0,0])
#    y = np.delete(y, [0, res-1], 1)
#    y = np.ndarray.flatten(y, order = 'F') 
#    y= np.append(y, [0,0])
#    z = np.delete(z, [0, res-1], 1)
#    z = np.ndarray.flatten(z, order = 'F') 
#    z= np.append(z, [1,-1])
    

    coordinates = np.vstack((x,y,z)).transpose() # coordinates is (#points, 3) shaped

    return coordinates, theta, phi





def pnt_in_cvex_hull(hull, pnt):
    '''
    Checks if `pnt` is inside the convex hull.
    '''
    new_hull = ConvexHull(np.concatenate((hull.points, [pnt])))
    if np.array_equal(new_hull.vertices, hull.vertices): 
        return True
    else:
        return False




def height(p, vertices, seeds, altitudes):
    max_dist = 0
    '''
    finding shortest edge
    '''
    for i in range(3):
        for j in range(i+1 , 4):
            dist = np.linalg.norm(vertices[i] - vertices[j])
            if dist > max_dist:
                max_dist = dist
                furthest = np.array([i, j])   
   
    '''
    reordering everything
    '''          
    vertices[[0, furthest[0]]] = vertices[[furthest[0], 0]]
    vertices[[1, furthest[1]]] = vertices[[furthest[1], 1]]
    
    seeds[[0,furthest[0]]] = seeds[[furthest[0], 0]]
    seeds[[1,furthest[1]]] = seeds[[furthest[1], 1]]
    
    altitudes[[0, furthest[0]]] = altitudes[[furthest[0], 0]]
    altitudes[[1, furthest[1]]] = altitudes[[furthest[1], 1]]
    
    '''
    creating new edge
    new altitude = average altitude + random(-0,05; 0,05)*sqrt(distance)
    '''
    v_new = (vertices[0]+vertices[1])/2
    s_new = (seeds[0]+seeds[1])/2
    random.seed(s_new)
    a_new = (altitudes[0]+ altitudes[1])/2 + 0.01*(random.random()- 0.5)*max_dist**1.5

    
    '''
    finding in which tetrahedon our point p is
    '''    
    tetra = np.copy(vertices)
    tetra[1] = v_new
    hull = ConvexHull(tetra)
    
    if pnt_in_cvex_hull(hull, p):
        vertices[1] = v_new
        seeds[1] = s_new
        altitudes[1] = a_new
    else:
        vertices[0] = v_new
        seeds[0] = s_new
        altitudes[0] = a_new 
    
    '''
    stop if resolution is great enough
    '''
    if max_dist < 0.001:
#        print(np.sum(altitudes)/4)
        return np.sum(altitudes)/4
    else:
#        print(vertices)
#        print(seeds)
#        print(altitudes)
        
        height(p, vertices, seeds, altitudes)
    return np.sum(altitudes)/4


def polar(polarcaps, sphere,alts, p_ice):
    minz = min(sphere[:,2])
    maxz = max(sphere[:,2])
    rowdivide = np.linspace(minz,maxz,100)
    rangez = abs(maxz - minz)
    
    iceval = max(alts)

    for i in range(len(alts)):
        if abs(maxz-sphere[i,2])/rangez < p_ice or abs(minz-sphere[i,2])/rangez < p_ice:
            alts[i] = iceval
    return alts

In [4]:
'''
Euler rotation matrices
'''
def positive(arg):
    return (arg + abs(arg))/2

def y_rotation(angle):
    Y = np.array([[np.cos(angle), 0, np.sin(angle)],[0, 1, 0],[-np.sin(angle), 0, np.cos(angle)]])
    return Y

def z_rotation(angle):
    Z = np.array([[np.cos(angle), -np.sin(angle), 0],[np.sin(angle), np.cos(angle), 0], [0, 0, 1]])
    return Z



R_equinox = z_rotation(alpha)
R_tilt = y_rotation(beta)
R_axial = np.matmul(R_equinox, R_tilt)


In [5]:
'''
Initialize transfromation matrices (edge-on, face-on)
'''
T = np.zeros((time_res, 2*res**2)) 


Tlamb = np.zeros((time_res, 2*res**2))
Tglint = np.zeros((time_res, 2*res**2))
Tmetal = np.zeros((time_res, 2*res**2))
Tseel = np.zeros((time_res, 2*res**2))

Tlambglint = np.zeros((time_res, 2*res**2))
Tlambglintseel = np.zeros((time_res, 2*res**2))
                          
T1 = T.copy()

'''
Observer

'''
o_vec = np.array([1, 0, 0])
o_vec_1 = np.array([0, 0, 1])

In [6]:
# '''
# Old T matrix
# '''
# for i in range(len(time_array)):
#     t = time_array[i]
#     r_vec = np.array([-np.cos(omega_year*t), -np.sin(omega_year*t), 0])
#     R_daily = z_rotation(omega_day*t)
#     daily_rotation = np.matmul(R_daily, R_axial)    
#     print(i,'/'+str(len(time_array)))
    
#     for j in range(2*res**2):
#         phi = math.floor(j/res)*delta_phi                  
#         theta = (j % res)*delta_theta
        
#         s_vec = np.array([np.cos(phi)*np.sin(theta), np.sin(phi)*np.sin(theta), np.cos(theta)])
#         s_vec_rotated = np.matmul(daily_rotation,s_vec)
        
#         r_s = np.dot(r_vec, s_vec_rotated)
#         s_o = np.dot(s_vec_rotated, o_vec)
#         s_o_1 = np.dot(s_vec_rotated, o_vec_1)
         
        
#         illuminated = positive(r_s)
#         visible = positive(s_o)
#         visible_1 = positive(s_o_1)
        
#         T[i][j] = illuminated*visible*np.sin(theta)*delta_Omega
#         T1[i][j] = illuminated*visible_1*np.sin(theta)*delta_Omega

In [7]:
#New T-matrix

'''
Compute matrix elements
'''
for i in range(len(time_array)):
    t = time_array[i]
    i_hat = np.array([np.cos(omega_year*t),np.sin(omega_year*t),0])
    r_hat = o_vec
    R_daily = z_rotation(omega_day*t)
    daily_rotation = np.matmul(R_axial,R_daily) 
    print(i,'/'+str(len(time_array)))
    
    for j in range(2*res**2):
        phi = math.floor(j/res)*delta_phi                  
        theta = (j % res)*delta_theta

        s_hat = np.array([np.cos(phi)*np.sin(theta),np.sin(phi)*np.sin(theta),np.cos(theta)])
        s_hat_rotated = np.matmul(daily_rotation,s_hat)

        cos_theta_i = np.dot(i_hat,s_hat_rotated)
        if cos_theta_i <-1 or cos_theta_i >1:
            cos_theta_i = round(cos_theta_i)
            
        theta_i = np.arccos(cos_theta_i)
        

        cos_theta_r = np.dot(r_hat,s_hat_rotated)
        if cos_theta_r <-1 or cos_theta_r >1:
            cos_theta_r = round(cos_theta_r)
            
        theta_r = np.arccos(cos_theta_r)

        p1 = 1/(np.linalg.norm(np.cross(i_hat,s_hat_rotated))+epsilon) #epsilon to ensure norm(p1)<1
        
        p2 = 1/(np.linalg.norm(np.cross(r_hat,s_hat_rotated))+epsilon)
        
        
        


        cos_psi = (np.dot(i_hat,r_hat)-np.dot(i_hat,s_hat_rotated)*np.dot(r_hat,s_hat_rotated))*p1*p2
        
        
        psi = np.arccos(cos_psi)

        Rs = abs((cos_theta_i-np.sqrt(nwater**2-np.sin(theta_i)**2))/(cos_theta_i+np.sqrt(nwater**2-np.sin(theta_i)**2)))**2
        Rp = abs(((nwater**2)*cos_theta_i-np.sqrt(nwater**2-np.sin(theta_i)**2))/((nwater**2)*cos_theta_i+np.sqrt(nwater**2-np.sin(theta_i)**2)))**2

        lune = observabledomain(cos_theta_i,cos_theta_r)*cos_theta_i*cos_theta_r

        
        Tlamb[i][j]  =                                                                                                                   1/np.pi*lune*np.sin(theta)*delta_Omega
        Tglint[i][j] =             (Rs+Rp)/2*deltapeak(theta_i-theta_r,sigma)*deltapeak(psi-np.pi,sigma)/(np.sin(theta_i)*cos_theta_i+epsilon)*lune*np.sin(theta)*delta_Omega
        Tmetal[i][j] =                     deltapeak(theta_i-theta_r,sigma)*deltapeak(psi-np.pi,sigma)/(np.sin(theta_i)*cos_theta_i+epsilon)*lune*np.sin(theta)*delta_Omega
        Tseel[i][j]  = (np.sin(omega_year*t)+(np.pi-omega_year*t)*np.cos(omega_year*t))/(cos_theta_i+cos_theta_r+epsilon)*2/(3*np.pi**2)*lune*np.sin(theta)*delta_Omega
        if i > len(time_array)/2:
            Tseel[i][j] = -Tseel[i][j]
        #T1[i][j] = illuminated*visible_1*np.sin(theta)*delta_Omega
#    print(i)

0 /8760
1 /8760
2 /8760
3 /8760
4 /8760
5 /8760
6 /8760
7 /8760
8 /8760
9 /8760
10 /8760
11 /8760
12 /8760
13 /8760
14 /8760
15 /8760
16 /8760
17 /8760
18 /8760
19 /8760
20 /8760
21 /8760
22 /8760
23 /8760
24 /8760
25 /8760
26 /8760
27 /8760
28 /8760
29 /8760
30 /8760
31 /8760
32 /8760
33 /8760
34 /8760
35 /8760
36 /8760
37 /8760
38 /8760
39 /8760
40 /8760
41 /8760
42 /8760
43 /8760
44 /8760
45 /8760
46 /8760
47 /8760
48 /8760
49 /8760
50 /8760
51 /8760
52 /8760
53 /8760
54 /8760
55 /8760
56 /8760
57 /8760
58 /8760
59 /8760
60 /8760
61 /8760
62 /8760
63 /8760
64 /8760
65 /8760
66 /8760
67 /8760
68 /8760
69 /8760
70 /8760
71 /8760
72 /8760
73 /8760
74 /8760
75 /8760
76 /8760
77 /8760
78 /8760
79 /8760
80 /8760
81 /8760
82 /8760
83 /8760
84 /8760
85 /8760
86 /8760
87 /8760
88 /8760
89 /8760
90 /8760
91 /8760
92 /8760
93 /8760
94 /8760
95 /8760
96 /8760
97 /8760
98 /8760
99 /8760
100 /8760
101 /8760
102 /8760
103 /8760
104 /8760
105 /8760
106 /8760
107 /8760
108 /8760
109 /8760
110 /8760


831 /8760
832 /8760
833 /8760
834 /8760
835 /8760
836 /8760
837 /8760
838 /8760
839 /8760
840 /8760
841 /8760
842 /8760
843 /8760
844 /8760
845 /8760
846 /8760
847 /8760
848 /8760
849 /8760
850 /8760
851 /8760
852 /8760
853 /8760
854 /8760
855 /8760
856 /8760
857 /8760
858 /8760
859 /8760
860 /8760
861 /8760
862 /8760
863 /8760
864 /8760
865 /8760
866 /8760
867 /8760
868 /8760
869 /8760
870 /8760
871 /8760
872 /8760
873 /8760
874 /8760
875 /8760
876 /8760
877 /8760
878 /8760
879 /8760
880 /8760
881 /8760
882 /8760
883 /8760
884 /8760
885 /8760
886 /8760
887 /8760
888 /8760
889 /8760
890 /8760
891 /8760
892 /8760
893 /8760
894 /8760
895 /8760
896 /8760
897 /8760
898 /8760
899 /8760
900 /8760
901 /8760
902 /8760
903 /8760
904 /8760
905 /8760
906 /8760
907 /8760
908 /8760
909 /8760
910 /8760
911 /8760
912 /8760
913 /8760
914 /8760
915 /8760
916 /8760
917 /8760
918 /8760
919 /8760
920 /8760
921 /8760
922 /8760
923 /8760
924 /8760
925 /8760
926 /8760
927 /8760
928 /8760
929 /8760
930 /8760


1592 /8760
1593 /8760
1594 /8760
1595 /8760
1596 /8760
1597 /8760
1598 /8760
1599 /8760
1600 /8760
1601 /8760
1602 /8760
1603 /8760
1604 /8760
1605 /8760
1606 /8760
1607 /8760
1608 /8760
1609 /8760
1610 /8760
1611 /8760
1612 /8760
1613 /8760
1614 /8760
1615 /8760
1616 /8760
1617 /8760
1618 /8760
1619 /8760
1620 /8760
1621 /8760
1622 /8760
1623 /8760
1624 /8760
1625 /8760
1626 /8760
1627 /8760
1628 /8760
1629 /8760
1630 /8760
1631 /8760
1632 /8760
1633 /8760
1634 /8760
1635 /8760
1636 /8760
1637 /8760
1638 /8760
1639 /8760
1640 /8760
1641 /8760
1642 /8760
1643 /8760
1644 /8760
1645 /8760
1646 /8760
1647 /8760
1648 /8760
1649 /8760
1650 /8760
1651 /8760
1652 /8760
1653 /8760
1654 /8760
1655 /8760
1656 /8760
1657 /8760
1658 /8760
1659 /8760
1660 /8760
1661 /8760
1662 /8760
1663 /8760
1664 /8760
1665 /8760
1666 /8760
1667 /8760
1668 /8760
1669 /8760
1670 /8760
1671 /8760
1672 /8760
1673 /8760
1674 /8760
1675 /8760
1676 /8760
1677 /8760
1678 /8760
1679 /8760
1680 /8760
1681 /8760
1682 /8760

2337 /8760
2338 /8760
2339 /8760
2340 /8760
2341 /8760
2342 /8760
2343 /8760
2344 /8760
2345 /8760
2346 /8760
2347 /8760
2348 /8760
2349 /8760
2350 /8760
2351 /8760
2352 /8760
2353 /8760
2354 /8760
2355 /8760
2356 /8760
2357 /8760
2358 /8760
2359 /8760
2360 /8760
2361 /8760
2362 /8760
2363 /8760
2364 /8760
2365 /8760
2366 /8760
2367 /8760
2368 /8760
2369 /8760
2370 /8760
2371 /8760
2372 /8760
2373 /8760
2374 /8760
2375 /8760
2376 /8760
2377 /8760
2378 /8760
2379 /8760
2380 /8760
2381 /8760
2382 /8760
2383 /8760
2384 /8760
2385 /8760
2386 /8760
2387 /8760
2388 /8760
2389 /8760
2390 /8760
2391 /8760
2392 /8760
2393 /8760
2394 /8760
2395 /8760
2396 /8760
2397 /8760
2398 /8760
2399 /8760
2400 /8760
2401 /8760
2402 /8760
2403 /8760
2404 /8760
2405 /8760
2406 /8760
2407 /8760
2408 /8760
2409 /8760
2410 /8760
2411 /8760
2412 /8760
2413 /8760
2414 /8760
2415 /8760
2416 /8760
2417 /8760
2418 /8760
2419 /8760
2420 /8760
2421 /8760
2422 /8760
2423 /8760
2424 /8760
2425 /8760
2426 /8760
2427 /8760

3082 /8760
3083 /8760
3084 /8760
3085 /8760
3086 /8760
3087 /8760
3088 /8760
3089 /8760
3090 /8760
3091 /8760
3092 /8760
3093 /8760
3094 /8760
3095 /8760
3096 /8760
3097 /8760
3098 /8760
3099 /8760
3100 /8760
3101 /8760
3102 /8760
3103 /8760
3104 /8760
3105 /8760
3106 /8760
3107 /8760
3108 /8760
3109 /8760
3110 /8760
3111 /8760
3112 /8760
3113 /8760
3114 /8760
3115 /8760
3116 /8760
3117 /8760
3118 /8760
3119 /8760
3120 /8760
3121 /8760
3122 /8760
3123 /8760
3124 /8760
3125 /8760
3126 /8760
3127 /8760
3128 /8760
3129 /8760
3130 /8760
3131 /8760
3132 /8760
3133 /8760
3134 /8760
3135 /8760
3136 /8760
3137 /8760
3138 /8760
3139 /8760
3140 /8760
3141 /8760
3142 /8760
3143 /8760
3144 /8760
3145 /8760
3146 /8760
3147 /8760
3148 /8760
3149 /8760
3150 /8760
3151 /8760
3152 /8760
3153 /8760
3154 /8760
3155 /8760
3156 /8760
3157 /8760
3158 /8760
3159 /8760
3160 /8760
3161 /8760
3162 /8760
3163 /8760
3164 /8760
3165 /8760
3166 /8760
3167 /8760
3168 /8760
3169 /8760
3170 /8760
3171 /8760
3172 /8760

...

5317 /8760
5318 /8760
5319 /8760
5320 /8760
5321 /8760
5322 /8760
5323 /8760
5324 /8760
5325 /8760
5326 /8760
5327 /8760
5328 /8760
5329 /8760
5330 /8760
5331 /8760
5332 /8760
5333 /8760
5334 /8760
5335 /8760
5336 /8760
5337 /8760
5338 /8760
5339 /8760
5340 /8760
5341 /8760
5342 /8760
5343 /8760
5344 /8760
5345 /8760
5346 /8760
5347 /8760
5348 /8760
5349 /8760
5350 /8760
5351 /8760
5352 /8760
5353 /8760
5354 /8760
5355 /8760
5356 /8760
5357 /8760
5358 /8760
5359 /8760
5360 /8760
5361 /8760
5362 /8760
5363 /8760
5364 /8760
5365 /8760
5366 /8760
5367 /8760
5368 /8760
5369 /8760
5370 /8760
5371 /8760
5372 /8760
5373 /8760
5374 /8760
5375 /8760
5376 /8760
5377 /8760
5378 /8760
5379 /8760
5380 /8760
5381 /8760
5382 /8760
5383 /8760
5384 /8760
5385 /8760
5386 /8760
5387 /8760
5388 /8760
5389 /8760
5390 /8760
5391 /8760
5392 /8760
5393 /8760
5394 /8760
5395 /8760
5396 /8760
5397 /8760
5398 /8760
5399 /8760
5400 /8760
5401 /8760
5402 /8760
5403 /8760
5404 /8760
5405 /8760
5406 /8760
5407 /8760

6062 /8760
6063 /8760
6064 /8760
6065 /8760
6066 /8760
6067 /8760
6068 /8760
6069 /8760
6070 /8760
6071 /8760
6072 /8760
6073 /8760
6074 /8760
6075 /8760
6076 /8760
6077 /8760
6078 /8760
6079 /8760
6080 /8760
6081 /8760
6082 /8760
6083 /8760
6084 /8760
6085 /8760
6086 /8760
6087 /8760
6088 /8760
6089 /8760
6090 /8760
6091 /8760
6092 /8760
6093 /8760
6094 /8760
6095 /8760
6096 /8760
6097 /8760
6098 /8760
6099 /8760
6100 /8760
6101 /8760
6102 /8760
6103 /8760
6104 /8760
6105 /8760
6106 /8760
6107 /8760
6108 /8760
6109 /8760
6110 /8760
6111 /8760
6112 /8760
6113 /8760
6114 /8760
6115 /8760
6116 /8760
6117 /8760
6118 /8760
6119 /8760
6120 /8760
6121 /8760
6122 /8760
6123 /8760
6124 /8760
6125 /8760
6126 /8760
6127 /8760
6128 /8760
6129 /8760
6130 /8760
6131 /8760
6132 /8760
6133 /8760
6134 /8760
6135 /8760
6136 /8760
6137 /8760
6138 /8760
6139 /8760
6140 /8760
6141 /8760
6142 /8760
6143 /8760
6144 /8760
6145 /8760
6146 /8760
6147 /8760
6148 /8760
6149 /8760
6150 /8760
6151 /8760
6152 /8760

6807 /8760
6808 /8760
6809 /8760
6810 /8760
6811 /8760
6812 /8760
6813 /8760
6814 /8760
6815 /8760
6816 /8760
6817 /8760
6818 /8760
6819 /8760
6820 /8760
6821 /8760
6822 /8760
6823 /8760
6824 /8760
6825 /8760
6826 /8760
6827 /8760
6828 /8760
6829 /8760
6830 /8760
6831 /8760
6832 /8760
6833 /8760
6834 /8760
6835 /8760
6836 /8760
6837 /8760
6838 /8760
6839 /8760
6840 /8760
6841 /8760
6842 /8760
6843 /8760
6844 /8760
6845 /8760
6846 /8760
6847 /8760
6848 /8760
6849 /8760
6850 /8760
6851 /8760
6852 /8760
6853 /8760
6854 /8760
6855 /8760
6856 /8760
6857 /8760
6858 /8760
6859 /8760
6860 /8760
6861 /8760
6862 /8760
6863 /8760
6864 /8760
6865 /8760
6866 /8760
6867 /8760
6868 /8760
6869 /8760
6870 /8760
6871 /8760
6872 /8760
6873 /8760
6874 /8760
6875 /8760
6876 /8760
6877 /8760
6878 /8760
6879 /8760
6880 /8760
6881 /8760
6882 /8760
6883 /8760
6884 /8760
6885 /8760
6886 /8760
6887 /8760
6888 /8760
6889 /8760
6890 /8760
6891 /8760
6892 /8760
6893 /8760
6894 /8760
6895 /8760
6896 /8760
6897 /8760

7552 /8760
7553 /8760
7554 /8760
7555 /8760
7556 /8760
7557 /8760
7558 /8760
7559 /8760
7560 /8760
7561 /8760
7562 /8760
7563 /8760
7564 /8760
7565 /8760
7566 /8760
7567 /8760
7568 /8760
7569 /8760
7570 /8760
7571 /8760
7572 /8760
7573 /8760
7574 /8760
7575 /8760
7576 /8760
7577 /8760
7578 /8760
7579 /8760
7580 /8760
7581 /8760
7582 /8760
7583 /8760
7584 /8760
7585 /8760
7586 /8760
7587 /8760
7588 /8760
7589 /8760
7590 /8760
7591 /8760
7592 /8760
7593 /8760
7594 /8760
7595 /8760
7596 /8760
7597 /8760
7598 /8760
7599 /8760
7600 /8760
7601 /8760
7602 /8760
7603 /8760
7604 /8760
7605 /8760
7606 /8760
7607 /8760
7608 /8760
7609 /8760
7610 /8760
7611 /8760
7612 /8760
7613 /8760
7614 /8760
7615 /8760
7616 /8760
7617 /8760
7618 /8760
7619 /8760
7620 /8760
7621 /8760
7622 /8760
7623 /8760
7624 /8760
7625 /8760
7626 /8760
7627 /8760
7628 /8760
7629 /8760
7630 /8760
7631 /8760
7632 /8760
7633 /8760
7634 /8760
7635 /8760
7636 /8760
7637 /8760
7638 /8760
7639 /8760
7640 /8760
7641 /8760
7642 /8760

8297 /8760
8298 /8760
8299 /8760
8300 /8760
8301 /8760
8302 /8760
8303 /8760
8304 /8760
8305 /8760
8306 /8760
8307 /8760
8308 /8760
8309 /8760
8310 /8760
8311 /8760
8312 /8760
8313 /8760
8314 /8760
8315 /8760
8316 /8760
8317 /8760
8318 /8760
8319 /8760
8320 /8760
8321 /8760
8322 /8760
8323 /8760
8324 /8760
8325 /8760
8326 /8760
8327 /8760
8328 /8760
8329 /8760
8330 /8760
8331 /8760
8332 /8760
8333 /8760
8334 /8760
8335 /8760
8336 /8760
8337 /8760
8338 /8760
8339 /8760
8340 /8760
8341 /8760
8342 /8760
8343 /8760
8344 /8760
8345 /8760
8346 /8760
8347 /8760
8348 /8760
8349 /8760
8350 /8760
8351 /8760
8352 /8760
8353 /8760
8354 /8760
8355 /8760
8356 /8760
8357 /8760
8358 /8760
8359 /8760
8360 /8760
8361 /8760
8362 /8760
8363 /8760
8364 /8760
8365 /8760
8366 /8760
8367 /8760
8368 /8760
8369 /8760
8370 /8760
8371 /8760
8372 /8760
8373 /8760
8374 /8760
8375 /8760
8376 /8760
8377 /8760
8378 /8760
8379 /8760
8380 /8760
8381 /8760
8382 /8760
8383 /8760
8384 /8760
8385 /8760
8386 /8760
8387 /8760

In [8]:
filename = 'NEWTmatrices_res'+str(res)+'_tres'+str(time_res)+'_a'+str(alpha)+'_b'+str(beta)+'.npz'
np.savez_compressed(filename, lamb = Tlamb, glint = Tglint, seel =Tseel)